In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.point_clouds.vae import VariationalAutoencoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from general_tools.in_out.basics import create_dir, delete_files_in_directory
from geo_tool import Point_Cloud

from general_tools.in_out.basics import files_in_subdirs
from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_type = raw_input('Give me the class type.').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]
experiment_name = 'bernoulli_1.0_full_shapes_' + class_syn_id + '_vae'

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/1024/', class_syn_id)
train_dir = osp.join(top_data_dir, 'OUT/models/')

train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

Give me the class type.chair


'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/bernoulli_1.0_full_shapes_03001627_vae'

In [5]:
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path)
full_pclouds, full_model_names, class_ids = pio.load_crude_point_clouds(file_names=full_file_names, n_threads=11)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )

6778 files containing complete point clouds were found.


In [6]:
train_data_, val_data_, test_data_ = pio.train_validate_test_split([full_pclouds, full_model_names],
                                                                   train_perc=0.9,
                                                                   validate_perc=0.0,
                                                                   test_perc=0.1,                                                                   
                                                                   seed=seed)

train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])

"1,2,3"


In [8]:
import tf_lab.models.point_net_based_AE as pnAE

def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


conf = Conf(n_input = [1024, 3],
            training_epochs = 1000,
            batch_size = 30,
            loss = 'bernoulli',
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = 5,
            learning_rate = 0.00002,
            saver_max_to_keep = 200,
            gauss_augment = {'mu': 0, 'sigma': 0.02},
            encoder = pnAE.encoder,
            decoder = pnAE.decoder,
            spatial_trans = True,             
            denoising = False,
            n_z = 512,
#             latent_vs_recon = 10,
            z_rotate = True
            )

reset_graph()
ae = VariationalAutoencoder(experiment_name, conf)
ae.train(train_data, conf)

('Epoch:', '0001', 'training time (minutes)=', '0.2978', 'loss=', '634.634871328')
('Epoch:', '0002', 'training time (minutes)=', '0.2643', 'loss=', '337.658206949')
('Epoch:', '0003', 'training time (minutes)=', '0.2626', 'loss=', '324.149007891')
('Epoch:', '0004', 'training time (minutes)=', '0.2616', 'loss=', '-346.330672466')
('Epoch:', '0005', 'training time (minutes)=', '0.2595', 'loss=', '-814.946782568')
('Epoch:', '0006', 'training time (minutes)=', '0.2638', 'loss=', '-1080.577683284')
('Epoch:', '0007', 'training time (minutes)=', '0.2623', 'loss=', '-865.912089193')
('Epoch:', '0008', 'training time (minutes)=', '0.2632', 'loss=', '-1328.381434605')
('Epoch:', '0009', 'training time (minutes)=', '0.2617', 'loss=', '-1578.963864331')
('Epoch:', '0010', 'training time (minutes)=', '0.2613', 'loss=', '-793.185370478')
('Epoch:', '0011', 'training time (minutes)=', '0.2606', 'loss=', '-1299.126570049')
('Epoch:', '0012', 'training time (minutes)=', '0.2612', 'loss=', '-1526.14

KeyboardInterrupt: 

In [28]:
# test_data.next_batch(conf.batch_size)


# ae.reconstruct()
# test_data.next_batch(conf.batch_size)[0][0].shape
# Point_Cloud(points=test_data.next_batch(conf.batch_size)[0][0]).plot()
# Point_Cloud(points=ae.reconstruct(test_data.next_batch)
# )[0]).plot
# Point_Cloud(points=ae.generate()[0]).plot()

ValueError: Cannot feed value of shape (30, 3072) for Tensor u'bernoulli_1.0_full_shapes_03001627_vae/Placeholder:0', which has shape '(?, 1024, 3)'